<img src="https://www.1min30.com/wp-content/uploads/2018/03/logo-NBA.jpg" style="height:150px">
<img src="https://www.bebasket.fr/_next/image?url=https%3A%2F%2Fcdn.bebasket.fr%2Fresources%2Fimages%2Fblog%2Fwembanyama-victor-nba-nuggets-2-1ef015d2-70ee-691e-99aa-09c56d1272d4.jpg&w=2048&q=75" style="height:150px">
<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center">Extraction des programmes de la NBA </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

- NBA:"https://www.playingnumbers.com/2019/12/how-to-get-nba-data-using-the-nba_api-python-module-beginner/"

In [67]:
# Packages nécessaires à installer
# pip install pandas
# pip install nba_api
# pip install pytz

import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder  # Pour accéder aux données de la NBA
from datetime import datetime, time  # Pour gérer les dates
import pytz  # Pour gérer les fuseaux horaires
from nba_api.stats.endpoints import TeamInfoCommon  # Pour accéder aux données de la NBA
from nba_api.stats.static import teams  # Pour extraire les équipes de la NBA

### EXTRATION DES DES PROGRAMMES DE MATCHS DE LA NBA SAISON 2024

In [198]:
def get_nba_games(season):
    # OBTENIR LES DONNÉES DES MATCHS POUR UNE SAISON SPÉCIFIQUE
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
    games = gamefinder.get_data_frames()[0]
    
    matches = []

    for index, game in games.iterrows():
        matchup = game['MATCHUP']
        team_name = game['TEAM_NAME']

        if 'vs.' in matchup:
            home_team = team_name
            away_team = matchup.split(' vs. ')[1]
        elif '@' in matchup:
            away_team = team_name
            home_team = matchup.split(' @ ')[0]
        else:
            continue

        full_datetime = datetime.strptime(game['GAME_DATE'], "%Y-%m-%d")

        # New condition: If full_datetime is before April 1st, do not change it
        if full_datetime < datetime(full_datetime.year, 4, 1):
            updated_datetime = full_datetime + timedelta(hours=1)
        else:
            # Check if the date is before or after the daylight saving time change
            if full_datetime < datetime(2023, 3, 26):
                # Subtract 10 minutes from the datetime
                updated_datetime = full_datetime + timedelta(minutes=-10)
            else:
                # Subtract 1 hour and 50 minutes from the datetime
                updated_datetime = full_datetime + timedelta(hours=-1, minutes=-50)

        formatted_game_datetime = updated_datetime.strftime("%Y-%m-%d %H:%M:%S")

        # AJOUTER LES DONNÉES DU MATCH À LA LISTE
        matches.append({
            "home_team": home_team,
            "away_team": away_team,
            "game_date": formatted_game_datetime
        })

    return matches

# EXEMPLE D'UTILISATION
season = "2024"
nba_games = get_nba_games(season)

# CRÉER UN DATAFRAME À PARTIR DES DONNÉES
df = pd.DataFrame(nba_games)

df.head(20)

,home_team,away_team,game_date
0,MEM,Memphis Grizzlies,2024-07-21 22:10:00
1,Miami Heat,MEM,2024-07-21 22:10:00
2,GSW,Golden State Warriors,2024-07-20 22:10:00
3,DET,Detroit Pistons,2024-07-20 22:10:00
4,ORL,Orlando Magic,2024-07-20 22:10:00
5,TOR,Toronto Raptors,2024-07-20 22:10:00
6,Miami Heat,GSW,2024-07-20 22:10:00
7,Brooklyn Nets,CHA,2024-07-20 22:10:00
8,Utah Jazz,DET,2024-07-20 22:10:00
9,Memphis Grizzlies,LAC,2024-07-20 22:10:00


In [200]:
# EXPORTER LE DATAFRAME EN FICHIER CSV
df.to_csv("nba_2024.csv", index=False)
print("Les données ont été exportées vers nba_games_2024.csv")

Les données ont été exportées vers nba_games_2024.csv


**Explications :**

Pour Identification des équipes à domicile et à l'extérieur  la methode consistait à dire :
- Si matchup contient "vs.", alors team_name est l'équipe à domicile et l'équipe après "vs." est l'équipe à l'extérieur.
- Si matchup contient "@", alors team_name est l'équipe à l'extérieur et l'équipe avant "@" est l'équipe à domicile.


### AUTRE METHODE D'AJUSTEMENT DE DATES ET HEURES

In [195]:
def get_nba_games(season):
    # OBTENIR LES DONNÉES DES MATCHS POUR UNE SAISON SPÉCIFIQUE
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season) ##car LeagueGameFinder ne fournit pas directement les heures des matchs.
    games = gamefinder.get_data_frames()[0]

    
    matches = []
    
    # FUSEAU HORAIRE DE LA NBA (HEURE DE L'EST DES ÉTATS-UNIS)
    nba_timezone = pytz.timezone('US/Eastern')
    
    # FUSEAU HORAIRE DE LA FRANCE (HEURE D'ÉTÉ)
    france_timezone = pytz.timezone('Europe/Paris')

    # HEURE PAR DÉFAUT POUR LES MATCHS (19:00, HEURE TYPIQUE POUR LE DÉBUT DES MATCHS DE LA NBA)
    default_game_time = time(19, 0)  # 7:00 PM

    for _, row in games.iterrows():
        matchup = row['MATCHUP']
        team_name = row['TEAM_NAME']
        
        # DÉTERMINER LES ÉQUIPES À DOMICILE ET À L'EXTÉRIEUR
        if ' @ ' in matchup:
            home_team = matchup.split(' @ ')[1]
            away_team = team_name
        elif ' vs. ' in matchup:
            home_team = team_name
            away_team = matchup.split(' vs. ')[1]
        else:
            print(f"Format de confrontation non reconnu: {matchup}")
            continue
        
        # EXTRAIRE LA DATE DU MATCH
        game_date = datetime.strptime(row['GAME_DATE'], "%Y-%m-%d")
        
        # COMBINER LA DATE ET L'HEURE PAR DÉFAUT
        game_datetime = datetime.combine(game_date, default_game_time)
        
        # LOCALISER À LA TIMEZONE NBA ET CONVERTIR À LA TIMEZONE FRANÇAISE
        game_datetime = nba_timezone.localize(game_datetime)
        game_datetime = game_datetime.astimezone(france_timezone)

        # FORMATAGE DATETIME AU FORMAT DÉSIRÉ
        formatted_game_datetime = game_datetime.strftime("%Y-%m-%d %H:%M:%S")

        # AJOUTER LES DONNÉES DU MATCH À LA LISTE
        matches.append({"home_team": home_team, "away_team": away_team, "game_date": formatted_game_datetime})

    return matches

# EXEMPLE D'UTILISATION
season = "2024"
nba_games = get_nba_games(season)

# CRÉER UN DATAFRAME À PARTIR DES DONNÉES
df = pd.DataFrame(nba_games) 
df.head(20)

,home_team,away_team,game_date
0,MIA,Memphis Grizzlies,2024-07-23 01:00:00
1,Miami Heat,MEM,2024-07-23 01:00:00
2,MIA,Golden State Warriors,2024-07-22 01:00:00
3,UTA,Detroit Pistons,2024-07-22 01:00:00
4,MIN,Orlando Magic,2024-07-22 01:00:00
5,SAS,Toronto Raptors,2024-07-22 01:00:00
6,Miami Heat,GSW,2024-07-22 01:00:00
7,Brooklyn Nets,CHA,2024-07-22 01:00:00
8,Utah Jazz,DET,2024-07-22 01:00:00
9,Memphis Grizzlies,LAC,2024-07-22 01:00:00


### EXTRATION DES EQUIPES DE LA NBA SAISON 2024

- source: "https://medium.com/@cristianvaldez85/nba-data-using-python-82ebc0e19398"

In [184]:
# Obtenir la liste des équipes NBA
nba_teams = teams.get_teams()

# Créer un dictionnaire des IDs des équipes
team_ids_dict = {team['full_name']: team['id'] for team in nba_teams}

# Créer un DataFrame vide pour stocker les données
df1 = pd.DataFrame()

# Parcourir les équipes et les ajouter au DataFrame
for team_name, team_id in team_ids_dict.items():
    # Obtenir les informations sur l'équipe
    team_info = TeamInfoCommon(team_id=team_id)
    # Récupérer le DataFrame contenant les informations de l'équipe
    df1_team = team_info.get_data_frames()[0]
    # Ajouter le nom de l'équipe au DataFrame
    df1_team['TeamName'] = team_name
    # Ajouter la saison au DataFrame
    df1_team['Season'] = '2023-24'
    # Concaténer les données de l'équipe actuelle avec le DataFrame principal
    df1 = pd.concat([df1, df1_team], ignore_index=True)

# Afficher les éléments de notre DataFrame
df1.head(10)

,TEAM_ID,SEASON_YEAR,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CONFERENCE,TEAM_DIVISION,TEAM_CODE,TEAM_SLUG,W,L,PCT,CONF_RANK,DIV_RANK,MIN_YEAR,MAX_YEAR,TeamName,Season
0,1610612737,2023-24,Atlanta,Hawks,ATL,East,Southeast,hawks,hawks,36,46,0.439,10,3,1949,2024,Atlanta Hawks,2023-24
1,1610612738,2023-24,Boston,Celtics,BOS,East,Atlantic,celtics,celtics,64,18,0.780,1,1,1946,2024,Boston Celtics,2023-24
2,1610612739,2023-24,Cleveland,Cavaliers,CLE,East,Central,cavaliers,cavaliers,48,34,0.585,4,2,1970,2024,Cleveland Cavaliers,2023-24
3,1610612740,2023-24,New Orleans,Pelicans,NOP,West,Southwest,pelicans,pelicans,49,33,0.598,7,2,2002,2024,New Orleans Pelicans,2023-24
4,1610612741,2023-24,Chicago,Bulls,CHI,East,Central,bulls,bulls,39,43,0.476,9,4,1966,2024,Chicago Bulls,2023-24
5,1610612742,2023-24,Dallas,Mavericks,DAL,West,Southwest,mavericks,mavericks,50,32,0.610,5,1,1980,2024,Dallas Mavericks,2023-24
6,1610612743,2023-24,Denver,Nuggets,DEN,West,Northwest,nuggets,nuggets,57,25,0.695,2,2,1976,2024,Denver Nuggets,2023-24
7,1610612744,2023-24,Golden State,Warriors,GSW,West,Pacific,warriors,warriors,46,36,0.561,10,5,1946,2024,Golden State Warriors,2023-24
8,1610612745,2023-24,Houston,Rockets,HOU,West,Southwest,rockets,rockets,41,41,0.500,11,3,1967,2024,Houston Rockets,2023-24
9,1610612746,2023-24,LA,Clippers,LAC,West,Pacific,clippers,clippers,51,31,0.622,4,1,1970,2024,Los Angeles Clippers,2023-24


In [5]:
# Exporter le DataFrame en fichier CSV
df.to_csv('nba_teams.csv', index=False)